In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
from tqdm import tqdm
from sklearn import svm
from sklearn import tree
from statistics import mode
%run Utilities.ipynb
%run Preprocessor.ipynb
%run LineSegmentor.ipynb
%run FeatureExtractor.ipynb
%run FeatureExtractor2.ipynb
%run Classifiers.ipynb

In [3]:
#Constants

#Path to generate the dataset at
testDatasetDirectory = "Data\data\\"

prep = Preprocessor()
extractor=LBP_Feature_Extractor()

In [4]:
# fig = plt.figure(figsize=(15,15))
# plt.imshow(x_test)
# plt.show

In [5]:
SVMcounter = 0
KNNcounter = 0
KNN5counter = 0
KMeanscounter = 0
RandomForest2counter = 0
RandomForest4counter = 0
RandomForest6counter = 0
RandomForest8counter = 0
RandomForest10counter = 0
counter = 0
for i in tqdm(range(400)):
    
    ##################################TRAINING############################################################
    ######Reading the inputs and labelling the training dataset######
    z = '%02d' % i
    #z = '01'
    x_train, y_train, x_test = readTestCase(testDatasetDirectory+z)
    x_train_gray = []
    x_train_binary = []
    x_train_original = []

    
    #####Preprocessing every image in the input dataset######
    for img in x_train:    
        gray, binary, original = prep.preprocessing_pipeline_image(img)
        x_train_gray.append(gray)
        x_train_binary.append(binary)
        x_train_original.append(original)
    
    
    
    #####Segmenting the dataset into lines#####
    x_train_segments =  np.empty(256)
    y_train_segments = []

    for i in range(len(x_train)):
        gray_lines, binary_lines, orig_lines = LineSegmentor(x_train_gray[i], x_train_binary[i], x_train_original[i]).segment()
        
        for j in range(len(binary_lines)):
#             if j>4:
#                 break
            lbpHist = computeLBPHist(binary_lines[j])
            x_train_segments = np.vstack((x_train_segments, lbpHist))
            
            ##Labelling the segmented dataset##
            if (i<2):
                y_train_segments.append(1)
            elif (i<4):
                y_train_segments.append(2)
            else:
                y_train_segments.append(3) 
    
    
    y_train_segments = np.asarray(y_train_segments)
    
    
    

    ######################################TEST###################################################### 
    
    #####Preprocessing every image in the testing dataset######
    x_test_gray, x_test_binary, x_test_original = prep.preprocessing_pipeline_image(x_test)
    
    
    #####Segmenting the dataset into lines#####
    ####And Calculating the features vector####
    gray_lines, binary_lines, orig_lines = LineSegmentor(x_test_gray, x_test_binary, x_test_original).segment()
    x_test_segments = np.empty([len(binary_lines), 256])
    
    
    for i in range(len(binary_lines)):
        lbpHist = computeLBPHist(binary_lines[i])
        x_test_segments[i] = lbpHist
        
       
    
    
    
    ####Classifying the test case#####
    SVMResults = SVMClassifier(x_train_segments[1:len(x_train_segments)], y_train_segments, x_test_segments)
    KNNResults = KNNClassifier(x_train_segments[1:len(x_train_segments)], y_train_segments, x_test_segments)
    RandomForest8Results = RandForestClassifier(x_train_segments[1:len(x_train_segments)], y_train_segments, x_test_segments, max_depth=8, random_state=0)

    results = SVMResults
    results = np.vstack((results,KNNResults))
    results = np.vstack((results,RandomForest8Results))
    results =  np.sum(results, axis = 0)
    
    prediction = np.zeros(1)
    prediction[0] = np.argmax(results) + 1
    
    if isCorrect(testDatasetDirectory, z, prediction):  
        counter += 1
    else:
        print(z)
        
        
    ####Calculating Accuracy#####
#    if isCorrect(testDatasetDirectory, z, SVMResults):
#        SVMcounter+=1
#    if isCorrect(testDatasetDirectory, z, KNNResults):
#        KNNcounter+=1 
#    if isCorrect(testDatasetDirectory, z, KNN5Results):
#        KNN5counter+=1 
#     if isCorrect(testDatasetDirectory, z, KMeansResults):
#         KMeanscounter+=1
#     if isCorrect(testDatasetDirectory, z, RandomForest2Results):
#         RandomForest2counter+=1 
#     if isCorrect(testDatasetDirectory, z, RandomForest4Results):
#         RandomForest4counter+=1     
#     if isCorrect(testDatasetDirectory, z, RandomForest6Results):
#         RandomForest6counter+=1    
#     if isCorrect(testDatasetDirectory, z, RandomForest8Results):
#         RandomForest8counter+=1     
#     if isCorrect(testDatasetDirectory, z, RandomForest10Results):
#         RandomForest10counter+=1            

 16%|█████████████▏                                                                   | 65/400 [06:55<35:52,  6.43s/it]

64


 42%|█████████████████████████████████▏                                              | 166/400 [18:35<25:54,  6.64s/it]

165


 54%|███████████████████████████████████████████▏                                    | 216/400 [24:44<23:52,  7.78s/it]

...

 95%|███████████████████████████████████████████████████████████████████████████▊    | 379/400 [44:18<02:21,  6.74s/it]

378


 99%|███████████████████████████████████████████████████████████████████████████████▍| 397/400 [46:20<00:19,  6.59s/it]

396


100%|████████████████████████████████████████████████████████████████████████████████| 400/400 [46:39<00:00,  7.00s/it]


In [6]:
# print(SVMcounter)
# print(KNNcounter)
# print(KNN5counter)
# print(KMeanscounter)
# print(RandomForest2counter)
# print(RandomForest4counter)
# print(RandomForest6counter)
# print(RandomForest8counter)
# print(RandomForest10counter)
print(counter)

387
